In [6]:
from bs4 import BeautifulSoup
import pathlib, os

import re
import requests

import folium
from IPython.display import display

# Geovisualisierung der in den Briefen erwähnten Orte

In [7]:
NOTEBOOK_PATH = pathlib.Path().resolve()
DATA_DIRECTORY = NOTEBOOK_PATH / "data" / "annotated"

Mit der folgenden Funktion werden die geonames-IDs zu GIS-Daten aufgelöst. Das wäre auch über die geonames-API möglich, die aber einen Account erfordert und bzgl. der Zugriffe stark limitiert ist. Daher wird hier der *Umweg* über Wikidata genommen.

In [8]:
def get_lat_lon_wikidata(geoname_id):
    query = f"""
    SELECT ?coord WHERE {{
      ?item wdt:P1566 "{geoname_id}".
      ?item wdt:P625 ?coord.
    }}
    LIMIT 1
    """

    url = "https://query.wikidata.org/sparql"
    headers = {
        "Accept": "application/sparql-results+json",
        "User-Agent": "PhilosophinnenImExilIII/1.0 (email@example.com)" 
    }
    r = requests.get(url, params={"query": query}, headers=headers)
    data = r.json()

    results = data.get("results", {}).get("bindings", [])
    if results:
        coord_str = results[0]["coord"]["value"] 
        
        match = re.match(r"Point\(([-\d\.]+) ([-\d\.]+)\)", coord_str)
        if match:
            lon = float(match.group(1))
            lat = float(match.group(2))
            return lat, lon
    return None, None

# Beispiel
lat, lon = get_lat_lon_wikidata("2643743")
print(lat, lon)

51.507222222 -0.1275


Die im ref-Element hinterlegten geonames-IDs werden aus dem XML extrahiert und zu GIS-Daten aufgelöst. Weil dafür mehrere Zugriffe auf Wikidata notwendig sind, dauert das ein paar Sekunden.

In [9]:
places = []

for file_name in os.listdir(DATA_DIRECTORY):
    if not file_name.endswith(".xml"):
        continue

    with open(DATA_DIRECTORY / file_name, encoding="utf-8") as f:
        soup = BeautifulSoup(f, "xml")

    for place in soup.find_all("placeName"):
        ref = place.get("ref")  
        if not ref or not ref.startswith("geo-"):
            continue

        geoname_id = ref.split("-")[1]
        lat, lon = get_lat_lon_wikidata(geoname_id)
        if lat is not None and lon is not None:
            places.append({
                "name": place.get_text(strip=True),
                "lat": lat,
                "lon": lon,
                "ref": ref
            })

Visualisierung der Daten mit der [Folium](https://python-visualization.github.io/folium/latest/)-Bibliothek. Die Ausgabe ist stark anpassbar, bspw. lassen sich die Marker mit verschiedenen Farben, Icons und auch einem Popup versehen, dass weiterführende Informationen enthält. 

* Wie ließe sich die Karte weiter anpassen, um informativer zu sein?
* Welche Fragestellungen lassen sich über so eine Geovisualisierung beantworten?

In [10]:
if places:
    avg_lat = sum(p["lat"] for p in places) / len(places)
    avg_lon = sum(p["lon"] for p in places) / len(places)
else:
    avg_lat, avg_lon = 0, 0

m = folium.Map(location=[avg_lat, avg_lon], zoom_start=4)

for p in places:
    folium.Marker(
        location=[p["lat"], p["lon"]],
        popup=f"{p['name']} ({p['ref']})",
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(m)

display(m)